In [1]:
import os

def check_subfolders_in_images(directory_path):
    """
    Checks the number of subfolders inside 'Images_001' for each main folder.

    Args:
    - directory_path (str): The main directory containing subfolders.

    Prints:
    - Number of subfolders inside 'Images_001' for each folder.
    """
    if os.path.exists(directory_path) and os.path.isdir(directory_path):
        folders = [f for f in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, f))]
        
        for folder in folders:
            image_001_path = os.path.join(directory_path, folder, "Images_001")
            if os.path.exists(image_001_path) and os.path.isdir(image_001_path):
                subfolders = [sf for sf in os.listdir(image_001_path) if os.path.isdir(os.path.join(image_001_path, sf))]
                print(f"Subfolders inside '{image_001_path}': {len(subfolders)}")
            else:
                print(f"No 'Images_001' folder found inside '{folder}'.")
    else:
        print(f"Directory '{directory_path}' does not exist or is not a directory.")

# Your directory path
directory_path = r"E:\2 Below Lake Waco Deep 28_8 Concentrated"
check_subfolders_in_images(directory_path)


Subfolders inside 'E:\2 Below Lake Waco Deep 28_8 Concentrated\2_38 Below Lake Waco Deep 28_8 Gal 9-17-21 Image1a\Images_001': 671
Subfolders inside 'E:\2 Below Lake Waco Deep 28_8 Concentrated\2_38 Below Lake Waco Deep 28_8 Gal 9-17-21 Image2a\Images_001': 387
Subfolders inside 'E:\2 Below Lake Waco Deep 28_8 Concentrated\2_38 Below Lake Waco Deep 28_8 Gal 9-17-21 Image3a\Images_001': 352
Subfolders inside 'E:\2 Below Lake Waco Deep 28_8 Concentrated\2_38 Below Lake Waco Deep 28_8 Gal 9-17-21 Image4a\Images_001': 277
Subfolders inside 'E:\2 Below Lake Waco Deep 28_8 Concentrated\2_38 Below Lake Waco Deep 28_8 Gal 9-17-21 Image5a\Images_001': 22
Subfolders inside 'E:\2 Below Lake Waco Deep 28_8 Concentrated\2_38 Below Lake Waco Deep 28_8 Gal 9-17-21 Image6a\Images_001': 789


In [ ]:
import os
import numpy as np
from tqdm import tqdm

try:
    from tensorflow.keras.preprocessing.image import load_img, img_to_array
except ImportError:
    print("TensorFlow is not installed. You can install it by running: pip install tensorflow")
    raise

def load_images_from_subfolders(directory_path):
    """
    Loads the first five images from each subfolder inside 'Images_001' for each main folder.
    Resizes the images to (28, 28, 3) for deep learning models.
    Pads with the last image if fewer than 5 images are available.
    Stores all subfolders' images in a single array and prints its shape.

    Args:
    - directory_path (str): The main directory containing subfolders.

    Prints:
    - Shape of the array containing resized images for all subfolders.
    """
    if os.path.exists(directory_path) and os.path.isdir(directory_path):
        folders = [f for f in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, f))]
        
        for folder in tqdm(folders, desc="Processing main folders"):
            image_001_path = os.path.join(directory_path, folder, "Images_001")
            if os.path.exists(image_001_path) and os.path.isdir(image_001_path):
                subfolders = [sf for sf in os.listdir(image_001_path) if os.path.isdir(os.path.join(image_001_path, sf))]
                
                # Array to store all images for the current main folder
                all_images = []

                for subfolder in tqdm(subfolders, desc=f"Processing subfolders in '{folder}'", leave=False):
                    subfolder_path = os.path.join(image_001_path, subfolder)
                    image_files = [f for f in os.listdir(subfolder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
                    
                    images = []
                    for image_file in image_files[:5]:
                        image_path = os.path.join(subfolder_path, image_file)
                        img = load_img(image_path, target_size=(28, 28))
                        img_array = img_to_array(img) / 255.0  # Normalize pixel values
                        images.append(img_array)

                    # Pad with the last image if there are fewer than 5
                    while len(images) < 5:
                        images.append(images[-1])

                    all_images.append(np.array(images))  # Add (5, 28, 28, 3) to the batch array

                # Convert to a numpy array of shape (num_subfolders, 5, 28, 28, 3)
                all_images_array = np.array(all_images)
                print(f"\nSubfolders inside '{image_001_path}': shape of images: {all_images_array.shape}")
            else:
                print(f"\nNo 'Images_001' folder found inside '{folder}'.")
    else:
        print(f"\nDirectory '{directory_path}' does not exist or is not a directory.")

# Your directory path
directory_path = r"E:\2 Below Lake Waco Deep 28_8 Concentrated"
load_images_from_subfolders(directory_path)


In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tqdm import tqdm

# Custom PatchEncoder layer
class PatchEncoder(tf.keras.layers.Layer):
    def __init__(self, num_patches, projection_dim, **kwargs):
        super(PatchEncoder, self).__init__(**kwargs)
        self.num_patches = num_patches
        self.projection_dim = projection_dim
        self.projection = tf.keras.Sequential(
            [
                tf.keras.layers.Conv2D(2, (3, 3), strides=(1, 1), activation='relu'),
                tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPooling2D((2, 2))),
                tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten()),
                tf.keras.layers.Dense(projection_dim),
            ]
        )
        self.position_embedding = tf.keras.layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def get_config(self):
        config = super().get_config()
        config.update({
            "num_patches": self.num_patches,
            "projection_dim": self.projection_dim,
        })
        return config

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

# Load model
model_path = r"E:\five_frame_model.h5"
custom_objects = {'PatchEncoder': PatchEncoder}

try:
    model = load_model(model_path, custom_objects=custom_objects)
    print("\nModel loaded successfully!")
except Exception as e:
    print(f"\nError loading model: {e}")
    raise

def load_images_from_subfolders(directory_path):
    """
    Loads the first five images from each subfolder inside 'Images_001' for each main folder.
    Resizes the images to (28, 28, 3) for deep learning models.
    Pads with the last image if fewer than 5 images are available.
    """
    if os.path.exists(directory_path) and os.path.isdir(directory_path):
        folders = [f for f in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, f))]
        
        for folder in tqdm(folders, desc="Processing main folders"):
            image_001_path = os.path.join(directory_path, folder, "Images_001")
            if os.path.exists(image_001_path) and os.path.isdir(image_001_path):
                subfolders = [sf for sf in os.listdir(image_001_path) if os.path.isdir(os.path.join(image_001_path, sf))]
                all_images = []

                for subfolder in tqdm(subfolders, desc=f"Processing subfolders in '{folder}'", leave=False):
                    subfolder_path = os.path.join(image_001_path, subfolder)
                    image_files = [f for f in os.listdir(subfolder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

                    images = []
                    for image_file in image_files[:5]:
                        image_path = os.path.join(subfolder_path, image_file)
                        img = load_img(image_path, target_size=(28, 28))
                        img_array = img_to_array(img) / 255.0  # Normalize pixel values
                        images.append(img_array)

                    # Pad with the last image if there are fewer than 5
                    while len(images) < 5:
                        images.append(images[-1])

                    all_images.append(np.array(images))  # Add (5, 28, 28, 3) to the batch array

                # Convert to a numpy array of shape (num_subfolders, 5, 28, 28, 3)
                all_images_array = np.array(all_images)
                print(f"\nSubfolders inside '{image_001_path}': shape of images: {all_images_array.shape}")

                # Make individual predictions for the current folder
                make_predictions(model, all_images_array, folder)
            else:
                print(f"\nNo 'Images_001' folder found inside '{folder}'.")
    else:
        print(f"\nDirectory '{directory_path}' does not exist or is not a directory.")

def make_predictions(model, data, folder_name):
    """
    Makes predictions on the given data and prints the count of each class for a specific folder.
    """
    print(f"\nMaking predictions for folder '{folder_name}'...")
    predictions = model.predict(data, batch_size=32, verbose=1)
    predicted_classes = np.argmax(predictions, axis=1)
    unique_classes, counts = np.unique(predicted_classes, return_counts=True)
    print(f"\nPredicted Class Distribution for '{folder_name}':")
    for cls, count in zip(unique_classes, counts):
        print(f"Class {cls}: {count}")

# Load images and make individual predictions
directory_path = r"E:\2 Below Lake Waco Deep 28_8 Concentrated"
#load_images_from_subfolders(directory_path)



Model loaded successfully!


In [ ]:
# Load images and make individual predictions
directory_path = r"G:\datasets\2.Veliger_Data\May 27 2022  11-18\15 Below Longhorn Dam River"
load_images_from_subfolders(directory_path)


Processing main folders:   0%|                                                                   | 0/6 [00:00<?, ?it/s]
Processing subfolders in '15_23 Below Longhorn Dam River 33 Minutes 12-3-21 Image1a':   0%|   | 0/1934 [00:00<?, ?it/s]
Processing subfolders in '15_23 Below Longhorn Dam River 33 Minutes 12-3-21 Image1a':   0%| | 2/1934 [00:00<02:09, 14.9
Processing subfolders in '15_23 Below Longhorn Dam River 33 Minutes 12-3-21 Image1a':   0%| | 4/1934 [00:00<02:33, 12.6
Processing subfolders in '15_23 Below Longhorn Dam River 33 Minutes 12-3-21 Image1a':   0%| | 6/1934 [00:00<02:27, 13.0
Processing subfolders in '15_23 Below Longhorn Dam River 33 Minutes 12-3-21 Image1a':   0%| | 8/1934 [00:00<02:51, 11.2
Processing subfolders in '15_23 Below Longhorn Dam River 33 Minutes 12-3-21 Image1a':   1%| | 10/1934 [00:00<03:32,  9.
Processing subfolders in '15_23 Below Longhorn Dam River 33 Minutes 12-3-21 Image1a':   1%| | 12/1934 [00:01<03:15,  9.
Processing subfolders in '15_23 Below Lo

Processing subfolders in '15_23 Below Longhorn Dam River 33 Minutes 12-3-21 Image1a':   7%| | 132/1934 [00:11<02:50, 10
Processing subfolders in '15_23 Below Longhorn Dam River 33 Minutes 12-3-21 Image1a':   7%| | 134/1934 [00:11<02:34, 11
Processing subfolders in '15_23 Below Longhorn Dam River 33 Minutes 12-3-21 Image1a':   7%| | 136/1934 [00:11<02:17, 13
Processing subfolders in '15_23 Below Longhorn Dam River 33 Minutes 12-3-21 Image1a':   7%| | 138/1934 [00:11<02:29, 12
Processing subfolders in '15_23 Below Longhorn Dam River 33 Minutes 12-3-21 Image1a':   7%| | 140/1934 [00:11<02:20, 12
Processing subfolders in '15_23 Below Longhorn Dam River 33 Minutes 12-3-21 Image1a':   7%| | 142/1934 [00:12<02:13, 13
Processing subfolders in '15_23 Below Longhorn Dam River 33 Minutes 12-3-21 Image1a':   7%| | 144/1934 [00:12<02:29, 12
Processing subfolders in '15_23 Below Longhorn Dam River 33 Minutes 12-3-21 Image1a':   8%| | 146/1934 [00:12<02:25, 12
Processing subfolders in '15_23 Below Lo

Processing subfolders in '15_23 Below Longhorn Dam River 33 Minutes 12-3-21 Image1a':  14%|▏| 266/1934 [00:21<01:49, 15
Processing subfolders in '15_23 Below Longhorn Dam River 33 Minutes 12-3-21 Image1a':  14%|▏| 269/1934 [00:21<01:35, 17
Processing subfolders in '15_23 Below Longhorn Dam River 33 Minutes 12-3-21 Image1a':  14%|▏| 271/1934 [00:21<01:35, 17
Processing subfolders in '15_23 Below Longhorn Dam River 33 Minutes 12-3-21 Image1a':  14%|▏| 273/1934 [00:21<01:36, 17
Processing subfolders in '15_23 Below Longhorn Dam River 33 Minutes 12-3-21 Image1a':  14%|▏| 276/1934 [00:21<01:34, 17
Processing subfolders in '15_23 Below Longhorn Dam River 33 Minutes 12-3-21 Image1a':  14%|▏| 278/1934 [00:22<01:35, 17
Processing subfolders in '15_23 Below Longhorn Dam River 33 Minutes 12-3-21 Image1a':  14%|▏| 280/1934 [00:22<01:49, 15
Processing subfolders in '15_23 Below Longhorn Dam River 33 Minutes 12-3-21 Image1a':  15%|▏| 283/1934 [00:22<01:39, 16
Processing subfolders in '15_23 Below Lo